# CALC FULL COLUMN QGPV, QGPVMEAN, QGPVP

In [1]:
#This notebook calculates QGPV, QGPV perturbation (QGPVP), and QGPV mean as in Melissa's code
#Last updated 05/20/20

In [2]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import metpy.calc as mpcalc
from metpy.units import units
import sys
from datetime import datetime, timedelta

from wrf import (getvar, interplevel, to_np, latlon_coords, get_cartopy,
                 cartopy_xlim, cartopy_ylim, extract_times, ALL_TIMES, interpz3d, vinterp)

In [3]:
#Define type of tile and desired pressure level
#fullphys_expdom or nolatentheat_expdom
filetype = "fullphys_expdom"                                   #CHANGE

In [4]:
# Open the NetCDF file
ncfile = Dataset("wrfout_"+filetype+".nc", "r")

# Extract the pressure, geopotential height, and wind variables
pressure = getvar(ncfile, "pressure",timeidx=ALL_TIMES)
z = getvar(ncfile, "z", timeidx=ALL_TIMES, units="m")
thta = getvar(ncfile, "th",timeidx=ALL_TIMES)
tmpk = getvar(ncfile, "tk",timeidx=ALL_TIMES)
ua = getvar(ncfile, "ua", timeidx=ALL_TIMES, units="kt")
va = getvar(ncfile, "va", timeidx=ALL_TIMES, units="kt")
height_agl = getvar(ncfile, "height_agl", timeidx = ALL_TIMES,units="m")

In [5]:
#Define constants
R = 287.05 #J kg^-1 K^-1
kappa=0.2856219 #Rd / cp
cp = 1004.7 #J kg^-1 K^-1
pref = 1000. #hPa
g = 9.80665 #gravity
omega = 7.292e-5 #Earth's rotation rate (s^-1)
a = 2.e7 / np.pi #Radius of the earth (m)
p0=1.e5
alpha=-1./5.255877
gamma = 0.0065 #lapse rate, (K/m)

In [6]:
#Define pressure level array
pb = 1000. #base pressure
pb_half = 975.
pt = 100. #top pressure
pt_half = 125.
p = np.linspace(pb,pt,19) #make array of size 19 with even spacing from base pressure to top pressure
#p_half = np.linspace(pb_half,pt_half,18)
delp = (p[0] - p[1]) * 100. #Convert to hPa

In [7]:
#Manually define phalf based on length of k and pressure values
phalf = np.zeros((len(p)-1))
for i in range(len(p)-1):
    phalf[i] = (p[i+1] + p[i]) / 2.

In [8]:
#Bounds for inversion
lat1 = 30. #Southern boundary
lat2 = 70. #Northern boundary
lon1 = 150. #Western boundary
lon2 = 250. #Eastern boundary
dlat = lat2 - lat1
dlon = lon2 - lon1
dellatstar = (np.pi/180.) * a
dellonstar = (np.pi/180.) * a
sigma1 = dellonstar / dellatstar
sigma2 = dellonstar / delp

In [9]:
#Get times from dataset
times = extract_times(ncfile, None)
count = np.count_nonzero(times)
#Set t, k, i, j indice lengths
tlen = len(times)
klen = len(p)
ilen = np.shape(z[:,:,:,:])[2] #This is really the j index (north-south index)
jlen = np.shape(z[:,:,:,:])[3] #This is really the i index (east-west index)

In [10]:
#Format times to be in standard format
dattimes = []
import pandas as pd
for i in range (0, count):
    dattimes.append(str(pd.Timestamp(times[i])))

## Calculate US Standard Atmosphere/Reference Atmosphere Variables

In [11]:
#Standard atmosphere geopotential, potential temperature, and stability function
def standard_atmosphere(p, p_half):
    g=9.8066
    kappa=0.2856219
    alpha=-1./5.255877
    beta=-6341.624
    gamma=.0065
    R=287.04
    cp = 1004.7
    p00=1000.
    nlevels = p.size
    nlevels_half = p_half.size

    Z = np.zeros(nlevels)
    phi = np.zeros(nlevels)
    T = np.zeros(nlevels)
    theta = np.zeros(nlevels)

    T_half = np.zeros(nlevels)
    theta_half = np.zeros(nlevels_half)
    dthetadp = np.zeros(nlevels_half)
    S = np.zeros(nlevels_half)

    for k in np.arange(nlevels):
        if (p[k] > 226.32):
            Z[k] = (288.15/gamma) * ( 1. - (1013.25/p[k])**alpha)
            T[k] = 288.15 - 0.0065*Z[k]
            theta[k] = T[k] * (1000./p[k])**(R/cp)
        else:
            T[k] = 216.65
            Z[k] = (11.e3+beta*np.log(p[k]/226.32))
            theta[k] = T[k] * (p00/p[k])**(R/cp)
                
    for k in np.arange(nlevels_half):
        if (p_half[k] > 226.32):
            T_half[k] = 288.15 - gamma*Z[k]
            theta_half[k] = T_half[k] * (p00/p_half[k])**(R/cp)
        else:
            T_half[k] = 216.65
            theta_half[k] = T_half[k] * (p00/p_half[k])**(R/cp)
    
    for k in np.arange(nlevels_half):
        dthetadp[k] = (theta[k+1] - theta[k])/(1.e2*(p[k+1]-p[k]))
        S[k] =  -R*(T_half[k]/theta_half[k])*dthetadp[k]/(p_half[k]*100.)
    return g*Z, T, S

In [12]:
#Calculate standard atmosphere geopotential (geop), potential temp (tmpk), and stability
SAgeop, SAtmpk, stability = standard_atmosphere(p, phalf)

In [14]:
hght = np.zeros((tlen, klen, ilen, jlen))
levels = [1000,950,900,850,800,750,700,650,600,550,500,450,400,350,300,250,200,150,100]
for t in range(tlen):
    hght[t,:,:,:] = vinterp(ncfile,z[t,:,:,:],'p',levels,extrapolate=True)

In [16]:
#Get lats and lons from wrf file
lat = getvar(ncfile, "lat")
lon = getvar(ncfile, "lon")

In [17]:
#Calculate lat and lon min and max for manually defining array lengths
latmax = float(np.max(lat))
latmin = float(np.min(lat))
lonmax = float(np.max(lon))
lonmin = float(np.min(lon))

In [18]:
#Redefine lat and lon arrays to be smaller size, 1 degree spacing
lat = np.arange(latmin,latmax,((latmax-latmin)/jlen))
lon = np.arange(lonmin,lonmax,((lonmax-lonmin)/ilen))

In [19]:
#Calculate Coriolis parameter
lenlat = len(lat)
cor = np.zeros((lenlat))
for j in range(lenlat):
    cor[j] = 2. * omega * np.sin(lat[j]*(np.pi / 180.)) # degrees to radians

In [20]:
#Calculate Rossby parameter (beta), variation in Coriolis force with respect to changing latitude
beta = np.zeros((lenlat))
for j in range(lenlat):
    beta[j] = (2. * omega / a) * np.cos(lat[j] * (np.pi / 180.))

In [21]:
#Calculate mean Coriolis Parameter weighted by latitude
fo = 0.
for j in range(lenlat):
    fo = fo + (cor[j] / (dlat+1)) #fo = fo + (cor[j] / (jlen-1))

## Calculate QGPV coefficients

In [22]:
cosPoint = np.zeros((lenlat))
cosPlus = np.zeros((lenlat)) 
cosPlushalf = np.zeros((lenlat)) 
cosMinus = np.zeros((lenlat)) 
cosMinushalf = np.zeros((lenlat))

for i in range(lenlat):
    cosPoint[i] = np.cos(lat[i] * (np.pi / 180.))
    cosPlus[i] = np.cos((lat[i] + 1.0) * np.pi / 180.)
    cosPlushalf[i] = np.cos((lat[i] + 0.5) * np.pi / 180.)
    cosMinus[i] = np.cos((lat[i] - 1.0) * np.pi / 180.)
    cosMinushalf[i] = np.cos((lat[i] - 0.5) * np.pi / 180.) 

In [23]:
#Coefficients on relative vorticity
A1_rel = np.zeros((lenlat))
A2_rel = np.zeros((lenlat))
A3_rel = np.zeros((lenlat))
A4_rel = np.zeros((lenlat))
A5_rel = np.zeros((lenlat))

for j in range(lenlat):
    A1_rel[j] = 1. / ((cosPoint[j] * dellonstar) ** 2.) # goes on phi(k,i-1,j)
    A2_rel[j] = cosMinushalf[j]/(cosPoint[j]*dellatstar ** 2.) # goes on phi(k,i,j-1)
    A3_rel[j] = (-2. / (cosPoint[j] * dellonstar) ** 2.) - (cosPlushalf[j] / (cosPoint[j] * dellatstar ** 2.)) - (cosMinushalf[j] / (cosPoint[j] * dellatstar ** 2.)) #goes on phi(k,i,j)
    A4_rel[j] = 1. / ((cosPoint[j] * dellonstar) ** 2.) # goes on phi(k,i+1,j) 
    A5_rel[j] = cosPlushalf[j] / (cosPoint[j] * dellatstar ** 2.) # goes on phi(k,i,j+1)
    
    

In [24]:
#Coefficients for QGPV
A1_qgpv = np.zeros((lenlat)) # goes on phi(t,j,i-1,k)
A2_qgpv = np.zeros((lenlat))
A2i_qgpv = np.zeros((lenlat)) # goes on phi(t,j-1,i,k)
A3_qgpv = np.zeros((lenlat, len(p))) # goes on phi(t,j,i,k) has parts in vert, meridional
A3i_qgpv = np.zeros((lenlat, len(p))) # goes on phi(t,j,i,k) has parts in vert, meridional
A4_qgpv = np.zeros((lenlat)) # goes on phi(t,j+1,i,k)
A4i_qgpv = np.zeros((lenlat)) # goes on phi(t,j+1,i,k)
A5_qgpv= np.zeros((lenlat)) # goes on phi(t,j,i+1,k) 
A6_qgpv = np.zeros((len(p))) # goes on phi(t,j,i,k-1) 
A7_qgpv = np.zeros((len(p))) # goes on phi(t,j,i,k+1) 
A8_qgpv = dellonstar ** 2. # goes on q-f 


for j in range(lenlat):
    A1_qgpv[j] = 1. / (fo * cosPoint[j] ** 2.) 
    A2_qgpv[j] = (sigma1 ** 2.) * cosMinus[j] / (fo * cosPoint[j])
    A2i_qgpv[j] = (sigma1 ** 2.) * cosMinushalf[j] / (fo * cosPoint[j]) 
    A4_qgpv[j] = ((sigma1 ** 2.) * cosPlus[j] / (fo * cosPoint[j]))
    A4i_qgpv[j] = ((sigma1 ** 2.) * cosPlushalf[j] / (fo * cosPoint[j])) 
    A5_qgpv[j] = 1. / (fo * cosPoint[j] ** 2.)
    
    for k in range(2,len(p)-1):
        A3_qgpv[j,k] = -1. * ((2. / (fo * cosPoint[j] ** 2.)) + ((sigma1 ** 2.) / fo) * ((cosPlus[j] / cosPoint[j]) + (cosMinus[j] / cosPoint[j])) + ((4. * fo * sigma2 ** 2.) * ((1. / stability[k-1]) + (1. / stability[k])))) 
        A3i_qgpv[j,k] = (-2. / (fo * cosPoint[j] ** 2.)) + (((sigma1 ** 2.) / fo) * ((-cosPlushalf[j] / cosPoint[j]) - (cosMinushalf[j] / cosPoint[j]))) + ((fo * sigma2 ** 2.) * ((-1. / stability[k-1]) - (1. / stability[k]))) 
        A6_qgpv[k] = (fo * sigma2 ** 2.) / stability[k-1] 
        A7_qgpv[k] = (fo * sigma2 ** 2.) / stability[k]


In [25]:
#Define arrays to store calculations
qgpv_tmp = np.zeros((tlen, klen, ilen, jlen))
qgpv = np.zeros((tlen, klen, ilen, jlen))
qgpvp = np.zeros((tlen, klen, ilen, jlen))
geop = np.zeros((tlen, klen, ilen, jlen))
geopper = np.zeros((tlen, klen, ilen, jlen))
rel_vor = np.zeros((tlen, klen, ilen, jlen))
strvor = np.zeros((tlen, klen, ilen, jlen))

In [26]:
#Reshape SAgeop for calculation
SAgeop1 = np.zeros((tlen, klen, ilen, jlen))
for t in range(tlen):
    for k in range(klen):
        SAgeop1[t,k,:,:] = SAgeop[k]

In [27]:
#Subtract standard atmosphere geopotential
for t in range(tlen):
    for k in range(klen):
        geop[t,k,:,:] = (g * hght[t,k,:,:]) - SAgeop1[t,k,:,:]
                

In [28]:
#Reshape SAtmpk for calculation
SAtmpk1 = np.zeros((tlen, klen, ilen, jlen))
for t in range(tlen):
    for k in range(klen):
        SAtmpk1[t,k,:,:] = SAtmpk[k]

In [29]:
#Define arrays for storing calculations
qgpvmean = np.zeros((klen, ilen, jlen)) #mean QGPV whole time period
geopmean = np.zeros((klen, ilen, jlen)) #mean geopotential
tmpkmean = np.zeros((klen, ilen, jlen)) #mean temp in Kelvin
tmpkmean_SArem = np.zeros((klen, ilen, jlen))
tmpk_SArem = np.zeros((tlen, klen, ilen, jlen))

In [30]:
#Subtract standard atmosphere temperature (from QGPV derivation, QG thermo equation)
for t in range(tlen):
    for k in range(klen):
        tmpk_SArem[t,k,:,:] = tmpk[t,k,:,:] - SAtmpk1[t,k,:,:]

In [31]:
#Calculate geopotential, temp (K) anomalies
#MEAN TMPK ANOMALIES TAKE AWHILE TO RUN
for k in range(klen):
    for i in range(ilen):
        for j in range(jlen):
            geopmean[k,i,j]= np.squeeze(np.mean(geop[0:,k,i,j])) 
            #tmpkmean[k,i,j]=np.squeeze(np.mean(tmpk[0:-1,k,i,j])) 
            #tmpkmean_SArem[k,i,j]=np.squeeze(np.mean(tmpk_SArem[0:-1,k,i,j])) 


In [32]:
#Calculate relative vorticity
for t in range(tlen):
    for k in range(1, klen-1):
        for i in range(1, ilen-1):
            for j in range(1, jlen-1): 
                rel_vor[t,k,i,j] = ((A1_rel[j] * geop[t,k,i-1,j]) + (A2_rel[j] * geop[t,k,i,j-1]) + (A3_rel[j] * geop[t,k,i,j]) + (A4_rel[j] * geop[t,k,i+1,j]) + (A5_rel[j] * geop[t,k,i,j+1])) / fo
                        

In [33]:
# calculate stretching vorticity  
str_vortot = np.zeros((tlen, klen, ilen, jlen))  
first_der = np.zeros((tlen, klen, ilen, jlen))

for t in range(tlen):
    for k in range(klen-1):
        first_der[t,k,:,:] = (1. / stability[k]) * ((geop[t,k,:,:] - geop[t,k+1,:,:]) / (delp))
            
            

In [34]:
#More stretching vorticity calculations
for t in range(tlen):
    for k in range(1,klen-1):
        str_vortot[t,k,:,:]= fo * (first_der[t,k-1,:,:] - first_der[t,k,:,:]) / (delp) 


In [35]:
# calculate qgpv for interior points 
for t in range(tlen):
    for k in range(1, klen-1):
        for i in range(1, ilen-1):
            for j in range(1, jlen-1):
                qgpv_tmp[t,k,i,j]=str_vortot[t,k,i,j]+rel_vor[t,k,i,j]+cor[j] #Using full Coriolis force, not beta plane or f plane approximations



In [36]:
# using hydrostatic relationship, set upper and lower
# boundaries using temperature 
for t in range(tlen):
    for i in range(ilen):
        for j in range(jlen):          
            qgpv_tmp[t,0,i,j] = ((-1.0*phalf[0]*100.)/R)*((geop[t,0,i,j]-geop[t,1,i,j])/(delp)) 
            qgpv_tmp[t,klen-1,i,j] = ((-1.0*phalf[klen-2]*100.)/R)*((geop[t,klen-3,i,j]-geop[t,klen-2,i,j])/delp)


In [37]:
#Assign qgpv values to array qgpv
qgpv = qgpv_tmp

In [38]:
# set sides equal to values just inside boundary as done for geopper 
for t in range(tlen):
    for k in range(klen):
        for i in range(ilen):
            qgpv[t,k,i,0] = qgpv[t,k,i,1]
            qgpv[t,k,i,jlen-1] = qgpv[t,k,i,jlen-2]
        for j in range(jlen): 
            qgpv[t,k,0,j] = qgpv[t,k,1,j]
            qgpv[t,k,ilen-1,j] = qgpv[t,k,ilen-2,j]


In [39]:
# set sides equal to values just inside boundary as done for geopper 
for t in range(tlen):
    for k in range(klen):
        for i in range(ilen):
            qgpv[t,k,i,0] = 0
            qgpv[t,k,i,jlen-1] = 0
        for j in range(jlen): 
            qgpv[t,k,0,j] = 0
            qgpv[t,k,ilen-1,j] = 0


In [40]:
# mean qgpv INCLUDING THETA TOP AND BOTTOM
#For whole time period
#qgpv[0:-1,k,i,j]

#For full_phys:
#Most rapid development (3 hour pressure tendencies >= -6mb/hr)
#12 UTC 11/26 to 00 UTC 11/27
#qgpv[8:12,k,i,j]

#For no_latent_heat:
#Most rapid development (3 hour pressure tendencies >= -6mb/hr)
#12 UTC 11/26 to 00 UTC 11/27
#qgpv[8:12,k,i,j]

qgpvmean_spec = np.zeros((klen, ilen, jlen)) #mean QGPV specified time period

for k in range(klen):
    for i in range(ilen):
        for j in range(jlen):
            qgpvmean[k,i,j] = np.squeeze(np.mean(qgpv[0:,k,i,j])) #Taking mean over whole time period
            qgpvmean_spec[k,i,j] = np.squeeze(np.mean(qgpv[8:12,k,i,j])) #Taking mean over specified time period


In [41]:
#Loop over u and v wind to get mean wind for QGPV MEAN whole time period plot
#This cell takes about 15 min to run but produces good results!

#uamean = np.zeros((klen, ilen, jlen))
#vamean = np.zeros((klen, ilen, jlen))
#for k in range(klen):
    #for i in range(ilen):
        #for j in range(jlen):
            #uamean[k,i,j] = np.squeeze(np.mean(ua[0:,k,i,j])) #Taking mean of u wind
            #vamean[k,i,j] = np.squeeze(np.mean(va[0:,k,i,j])) #Taking mean of v wind

In [42]:
#Loop over u and v wind to get mean wind for QGPV MEAN specified plot
#This cell takes about 15 min to run but produces good results!

#uamean_spec = np.zeros((klen, ilen, jlen))
#vamean_spec = np.zeros((klen, ilen, jlen))
#for k in range(klen):
    #for i in range(ilen):
        #for j in range(jlen):
            #uamean_spec[k,i,j] = np.squeeze(np.mean(ua[8:12,k,i,j])) #Taking mean of u wind
            #vamean_spec[k,i,j] = np.squeeze(np.mean(va[8:12,k,i,j])) #Taking mean of v wind

In [43]:
#Calculate QGPV anomalies, qgpvp, and geopotential anomalies
geopp = np.zeros((tlen,klen, ilen, jlen))
for t in range(tlen):
    qgpvp[t,:,:,:] = qgpv[t,:,:,:] - qgpvmean[:,:,:]
    geopp[t,:,:,:] = geop[t,:,:,:] - geopmean[:,:,:]
        

In [51]:
#START HERE TO RUN PLOTTING SEQUENCE

#Desired pressure (levtitle) and corresponding k value
levtitle = '300'                                                   #CHANGE                                                              #CHANGE
if levtitle == '850':
    lev = 3
if levtitle == '300':
    lev = 14

In [52]:
#import pickle
# Getting back the objects:
#with open('qgpv_vars_'+levtitle+'_'+filetype+'.pkl', 'rb') as f:
    #qgpv, qgpvp, qgpvmean, geop, geopp, geopmean, uamean, vamean, uamean_spec, vamean_spec, SAgeop, SAtmpk, stability = pickle.load(f)

In [53]:
#PLOTTING FOR QGPVMEAN whole time period
# Get the map projection information
cart_proj = get_cartopy(wrfin=ncfile)
    
lats, lons = latlon_coords(z)
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)
    
# Create the figure
fig = plt.figure(figsize=(12,9))
ax = plt.axes(projection=cart_proj)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
ax.add_feature(states, linewidth=0.5, edgecolor="black")
ax.coastlines('50m', linewidth=0.8)

leveof_qgpv = np.arange(-100, 100, 1)
tmp1 = 10. ** 4. * np.squeeze(qgpvmean[lev,:,:])
tmp1[tmp1 > 50.] = np.nan
tmp1[tmp1 < -50.] = np.nan

tmp2 = np.squeeze(uamean[lev,:,:])
tmp3 = np.squeeze(vamean[lev,:,:])

qgcs = plt.pcolormesh(to_np(lons), to_np(lats), to_np(tmp1),
                                cmap='bwr', vmin=-10., vmax=15.,
                                transform=crs.PlateCarree())
plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)

# Add the wind barbs, only plotting every 15th data point.             #UNCOMMENT
plt.barbs(to_np(lons[::15,::15]), to_np(lats[::15,::15]),
        to_np(tmp2[::15, ::15]), to_np(tmp3[::15, ::15]),
        transform=crs.PlateCarree(), length=5)

# Set the map bounds
ax.set_xlim(cartopy_xlim(z))
ax.set_ylim(cartopy_ylim(z))

ax.gridlines()

ax.set_title('WRF '+levtitle+'-hPa Mean QGPV (Scaled $10^4$) ($s^{-1}$) and Wind Barbs (kt) \n Valid From '+dattimes[0]+' UTC to '+dattimes[-1]+' UTC')
    
plt.savefig('plots/qgpvmean/QGPVMEAN_'+levtitle+'_'+filetype+'_'+dattimes[0]+'- '+dattimes[-1]+'.pdf')
plt.close()

In [54]:
#PLOTTING FOR QGPVMEAN specified time period
# Get the map projection information
cart_proj = get_cartopy(wrfin=ncfile)
    
lats, lons = latlon_coords(z)
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)
    
# Create the figure
fig = plt.figure(figsize=(12,9))
ax = plt.axes(projection=cart_proj)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
ax.add_feature(states, linewidth=0.5, edgecolor="black")
ax.coastlines('50m', linewidth=0.8)

leveof_qgpv = np.arange(-100, 100, 1)
tmp1 = 10. ** 4. * np.squeeze(qgpvmean_spec[lev,:,:])
tmp1[tmp1 > 50.] = np.nan
tmp1[tmp1 < -50.] = np.nan

tmp2 = np.squeeze(uamean_spec[lev,:,:])
tmp3 = np.squeeze(vamean_spec[lev,:,:])

qgcs = plt.pcolormesh(to_np(lons), to_np(lats), to_np(tmp1),
                                cmap='bwr', vmin=-10., vmax=15.,
                                transform=crs.PlateCarree())
plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)

# Add the wind barbs, only plotting every 15th data point.             #UNCOMMENT
plt.barbs(to_np(lons[::15,::15]), to_np(lats[::15,::15]),
        to_np(tmp2[::15, ::15]), to_np(tmp3[::15, ::15]),
        transform=crs.PlateCarree(), length=5)

# Set the map bounds
ax.set_xlim(cartopy_xlim(z))
ax.set_ylim(cartopy_ylim(z))

ax.gridlines()

ax.set_title('WRF '+levtitle+'-hPa Mean QGPV (Scaled $10^4$) ($s^{-1}$) and Wind Barbs (kt) \n Valid From '+dattimes[8]+' UTC to '+dattimes[12]+' UTC')
    
plt.savefig('plots/qgpvmean/QGPVMEAN_'+levtitle+'_'+filetype+'_'+dattimes[8]+'- '+dattimes[12]+'.pdf')
plt.close()

In [55]:
#PLOTTING FOR QGPVP (QGPV perturbation)
for i in range (0,count):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    lats, lons = latlon_coords(z)
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)
    
    #Plot qgpvmean
    leveof_qgpv = np.arange(-10,11, 1)
    tmp1 = 10. ** 4. * np.squeeze(qgpvp[i,lev,:,:])
    tmp1[tmp1 > 50.] = np.nan
    tmp1[tmp1 < -50.] = np.nan
   
    qgcs = plt.pcolormesh(to_np(lons), to_np(lats), to_np(tmp1),
                                   cmap='bwr', vmin=-10, vmax=10,
                                   transform=crs.PlateCarree())
    plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    # Add the wind barbs, only plotting every 15th data point.
    plt.barbs(to_np(lons[::15,::15]), to_np(lats[::15,::15]),
          to_np(ua[i,lev, ::15, ::15]), to_np(va[i,lev, ::15, ::15]),
          transform=crs.PlateCarree(), length=5)
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines()

    ax.set_title('WRF '+levtitle+'-hPa QGPV Perturbation (Scaled $10^4$) (1/s) and Wind Barbs (kt) \n Valid Time: {} UTC'.format(dattimes[i]))
    
    plt.savefig('plots/qgpvp/QGPVP_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

Working on time:  2019-11-25 12:00:00
Working on time:  2019-11-25 15:00:00
Working on time:  2019-11-25 18:00:00
Working on time:  2019-11-25 21:00:00
Working on time:  2019-11-26 00:00:00
Working on time:  2019-11-26 03:00:00
Working on time:  2019-11-26 06:00:00
Working on time:  2019-11-26 09:00:00
Working on time:  2019-11-26 12:00:00
Working on time:  2019-11-26 15:00:00
Working on time:  2019-11-26 18:00:00
Working on time:  2019-11-26 21:00:00
Working on time:  2019-11-27 00:00:00
Working on time:  2019-11-27 03:00:00
Working on time:  2019-11-27 06:00:00
Working on time:  2019-11-27 09:00:00
Working on time:  2019-11-27 12:00:00
Working on time:  2019-11-27 15:00:00
Working on time:  2019-11-27 18:00:00
Working on time:  2019-11-27 21:00:00
Working on time:  2019-11-28 00:00:00


In [56]:
#Save important values into pickle file
import pickle
with open('qgpv_vars_'+levtitle+'_'+filetype+'.pkl', 'wb') as f:
    pickle.dump([qgpv, qgpvp, qgpvmean, geop, geopp, geopmean, uamean, vamean, uamean_spec, vamean_spec, SAgeop, SAtmpk, stability], f)